# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In order to find which preservative has the strongest effect, we will use the entire set of data to model a classification algorithm (Decision Tree or Random Forest), and look at the 'coefficients' for each features (that is, for each preservative)

### Load Spark and Data

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treeproject').getOrCreate()

In [34]:
from pyspark.sql.functions import format_number

read_data = spark.read.csv('dog_food.csv',inferSchema=True, header=True)

data = read_data.select([
    read_data['A'].cast('integer'),
    read_data['B'].cast('integer'),
    read_data['C'].cast('integer'),
    read_data['D'].cast('integer'),
    read_data['Spoiled'].cast('integer')
])

data.show(5)

+---+---+---+---+-------+
|  A|  B|  C|  D|Spoiled|
+---+---+---+---+-------+
|  4|  2| 12|  3|      1|
|  5|  6| 12|  7|      1|
|  6|  2| 13|  6|      1|
|  4|  2| 12|  1|      1|
|  4|  2| 12|  3|      1|
+---+---+---+---+-------+
only showing top 5 rows



### Clean data

In [35]:
# list number of NANs or NULLs in each column
from pyspark.sql.functions import count, when, isnan, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+---+---+---+-------+
|  A|  B|  C|  D|Spoiled|
+---+---+---+---+-------+
|  0|  0|  0|  0|      0|
+---+---+---+---+-------+



In [36]:
# nothing to drop
cleaned_data = data

In [37]:
# check stats of each column with a pretty printed describe()

from pyspark.sql.functions import format_number

cdd = cleaned_data.describe()
cdd.select([cdd['summary'],
            format_number(cdd['A'].cast('double'), 2).alias('A'),
            format_number(cdd['B'].cast('double'), 2).alias('B'),
            format_number(cdd['C'].cast('double'), 2).alias('C'),
            format_number(cdd['D'].cast('double'), 2).alias('D'),
            format_number(cdd['Spoiled'].cast('double'), 2).alias('Spoiled'),
           ]).show()

+-------+------+------+------+------+-------+
|summary|     A|     B|     C|     D|Spoiled|
+-------+------+------+------+------+-------+
|  count|490.00|490.00|490.00|490.00| 490.00|
|   mean|  5.53|  5.50|  9.13|  5.58|   0.29|
| stddev|  2.95|  2.85|  2.06|  2.85|   0.45|
|    min|  1.00|  1.00|  5.00|  1.00|   0.00|
|    max| 10.00| 10.00| 14.00| 10.00|   1.00|
+-------+------+------+------+------+-------+



### Format Data for Spark

In [38]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [44]:
assembler = VectorAssembler(
    inputCols=['A','B','C','D'],
    outputCol="features")

prepared_data = assembler.transform(data).select(['features', 'Spoiled'])

In [45]:
prepared_data.show(5)

+------------------+-------+
|          features|Spoiled|
+------------------+-------+
|[4.0,2.0,12.0,3.0]|      1|
|[5.0,6.0,12.0,7.0]|      1|
|[6.0,2.0,13.0,6.0]|      1|
|[4.0,2.0,12.0,1.0]|      1|
|[4.0,2.0,12.0,3.0]|      1|
+------------------+-------+
only showing top 5 rows



no need to split into train and test because for this problem we will be using the entire set of data to fit the model and get the 'coefficients'

### Random Forest Classifier

In [52]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier

In [53]:
dtc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features', numTrees=100)

In [54]:
dtc_model = dtc.fit(prepared_data)
rfc_model = rfc.fit(prepared_data)

In [55]:
dtc_model.featureImportances

SparseVector(4, {0: 0.0026, 1: 0.0089, 2: 0.9686, 3: 0.0199})

In [56]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0187, 1: 0.0202, 2: 0.9348, 3: 0.0264})

The C preservative has clearly the strongest effect!